In [49]:
from dotenv import load_dotenv
import io
import base64
import re
import google.generativeai as genai
from PIL import Image
import gradio as gr
import os

In [50]:
load_dotenv()
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
if not GEMINI_API_KEY:
    raise ValueError("GEMINI_API_KEY not found in .env file")
genai.configure(api_key=GEMINI_API_KEY)


In [51]:
def image_to_base64(image_pil):
    """Converts a PIL image to a base64 encoded string."""
    buffered = io.BytesIO()
    image_pil.save(buffered, format="JPEG")
    return base64.b64encode(buffered.getvalue()).decode("utf-8")


def extract_code_blocks(text):
    """Extracts HTML and CSS code blocks from markdown-formatted text."""
    html_pattern = r'```html\s*(.*?)```'
    css_pattern = r'```css\s*(.*?)```'
    
    html_match = re.search(html_pattern, text, re.DOTALL)
    css_match = re.search(css_pattern, text, re.DOTALL)
    
    html_code = html_match.group(1).strip() if html_match else ""
    css_code = css_match.group(1).strip() if css_match else ""
    
    # If no separate CSS block, check if CSS is embedded in HTML
    if not css_code and '<style>' in html_code:
        style_match = re.search(r'<style>(.*?)</style>', html_code, re.DOTALL)
        if style_match:
            css_code = style_match.group(1).strip()
    
    return html_code, css_code

In [52]:
def combine_css(html_content, css_content, css_filename="style.css"):
    
    link_tag = '<link rel="stylesheet" href="style.css">'
    if link_tag not in html_content and not css_content:
        return html_content
    style_block = f"<style>\n{css_content}\n</style>"
    # Replace the <link> tag with the <style> block.
    modified_html = html_content.replace(link_tag, style_block, 1)

    return modified_html

In [53]:
def process_images(template_image, img1, img2, img3, img4, img5):
    """Processes the template and user images to generate HTML and CSS code."""
    if template_image is None:
        return "Please upload a template image.", "", "", ""

    try:
        prompt = """Analyze the following template image and generate clean, semantic HTML and CSS code that replicates its structure and design.

Requirements:
- Generate modern, responsive HTML5 code
- Create clean CSS with appropriate styling
- Use placeholder images with src="placeholder1.jpg", "placeholder2.jpg", etc.
- Make the layout responsive
- Include all necessary HTML structure (doctype, html, head, body tags)
- Put CSS in a separate code block
- Link the CSS file in the HTML head with <link rel="stylesheet" href="style.css">

Format your response with:
```html
[HTML code here]
```

```css
[CSS code here]
```
"""

        model = genai.GenerativeModel('gemini-2.5-flash')
        response = model.generate_content([prompt, template_image])
        
        generated_text = response.text
        html_code, css_code = extract_code_blocks(generated_text)

        if not html_code:
            return "Failed to generate HTML code. Please try again.", generated_text, "", ""

        # Replace placeholder images with user images
        processed_html = html_code
        user_images = [img1, img2, img3, img4, img5]
        
        for i, user_image in enumerate(user_images, start=1):
            if user_image is not None:
                #base64_image = image_to_base64(Image.open(user_image))
                base64_image = image_to_base64(user_image)
                data_uri = f'data:image/jpeg;base64,{base64_image}'
                
                # Replace various placeholder patterns
                processed_html = re.sub(f'src="placeholder{i}.jpg"', f'src="{data_uri}"', processed_html)
                processed_html = re.sub(f'src="placeholder-{i}.jpg"', f'src="{data_uri}"', processed_html)
                processed_html = re.sub(f'src="image{i}.jpg"', f'src="{data_uri}"', processed_html)
                
                # Replace first placeholder.com for first image
                if i == 1:
                    processed_html = re.sub(
                        r'src="https://via\.placeholder\.com/[^"]*"',
                        f'src="{data_uri}"',
                        processed_html,
                        count=1
                    )

        # Combine HTML and CSS for rendering
        #rendered_html = f"<style>{css_code}</style>{processed_html}"
        rendered_html = combine_css(processed_html, css_code)

        return "Code generated successfully!", processed_html, css_code, rendered_html

    except Exception as e:
        return f"An error occurred: {str(e)}", "", "", ""

In [54]:
iface = gr.Interface(
    fn=process_images,
    inputs=[
        gr.Image(type="pil", label="Template Image"),
        gr.Image(type="pil", label="User Image 1 (Optional)"),
        gr.Image(type="pil", label="User Image 2 (Optional)"),
        gr.Image(type="pil", label="User Image 3 (Optional)"),
        gr.Image(type="pil", label="User Image 4 (Optional)"),
        gr.Image(type="pil", label="User Image 5 (Optional)")
    ],
    outputs=[
        gr.Textbox(label="Status"),
        gr.Code(label="HTML Code", language="html"),
        gr.Code(label="CSS Code", language="css"),
        gr.HTML(label="Preview")
    ],
    title="🎨 Image to HTML/CSS Converter",
    description="Upload a template image (screenshot, mockup, or design) and optionally up to 5 images to replace placeholders.\n\n",
    allow_flagging="never"
)

iface.launch(share=True)

Running on local URL:  http://127.0.0.1:7866
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://944b408afce8ff58e7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [55]:
print("Available models:")
for model in genai.list_models():
    if 'generateContent' in model.supported_generation_methods:
        print(f"  - {model.name}")

Available models:
  - models/gemini-2.5-pro-preview-03-25
  - models/gemini-2.5-flash-preview-05-20
  - models/gemini-2.5-flash
  - models/gemini-2.5-flash-lite-preview-06-17
  - models/gemini-2.5-pro-preview-05-06
  - models/gemini-2.5-pro-preview-06-05
  - models/gemini-2.5-pro
  - models/gemini-2.0-flash-exp
  - models/gemini-2.0-flash
  - models/gemini-2.0-flash-001
  - models/gemini-2.0-flash-lite-001
  - models/gemini-2.0-flash-lite
  - models/gemini-2.0-flash-lite-preview-02-05
  - models/gemini-2.0-flash-lite-preview
  - models/gemini-2.0-pro-exp
  - models/gemini-2.0-pro-exp-02-05
  - models/gemini-exp-1206
  - models/gemini-2.0-flash-thinking-exp-01-21
  - models/gemini-2.0-flash-thinking-exp
  - models/gemini-2.0-flash-thinking-exp-1219
  - models/gemini-2.5-flash-preview-tts
  - models/gemini-2.5-pro-preview-tts
  - models/learnlm-2.0-flash-experimental
  - models/gemma-3-1b-it
  - models/gemma-3-4b-it
  - models/gemma-3-12b-it
  - models/gemma-3-27b-it
  - models/gemma-3n-